In [1]:
from pathlib import Path
import json
import pandas as pd

runs_dir = Path("../runs")

def latest_run_dir(runs_dir: Path) -> Path:
    if not runs_dir.exists():
        raise FileNotFoundError(f"Runs directory not found: {runs_dir}")
    candidates = [p for p in runs_dir.iterdir() if p.is_dir()]
    if not candidates:
        raise FileNotFoundError(f"No run directories found under: {runs_dir}")
    return sorted(candidates, key=lambda p: p.stat().st_mtime, reverse=True)[0]

run_dir = latest_run_dir(runs_dir)
print(f"Using run: {run_dir.name}")

instances_path = run_dir / "instances.json"
if not instances_path.exists():
    raise FileNotFoundError(f"Missing instances.json at: {instances_path}")

with instances_path.open("r", encoding="utf-8") as f:
    data = json.load(f)

instances = data.get("instances") or []
df = pd.json_normalize(instances)
print(df.shape)

Using run: 20260129_014857
(85, 70)


In [2]:
df_recruiting = df[df["canonical_process"] == "recruiting"].copy()
len(df), len(df_recruiting)


(85, 31)

In [3]:
round((df_recruiting["canonical_current_step_id"].notna()).mean(), 2)


np.float64(0.68)

In [4]:
has_raw_step = df_recruiting["state.step"].notna() & (df_recruiting["state.step"].astype(str).str.strip() != "")
has_canonical = df_recruiting["canonical_current_step_id"].notna()

pd.DataFrame({
    "raw_step_present_pct": [has_raw_step.mean()],
    "canonical_step_present_pct": [has_canonical.mean()],
    "raw_but_not_canonical_pct": [(has_raw_step & ~has_canonical).mean()],
    "neither_raw_nor_canonical_pct": [(~has_raw_step & ~has_canonical).mean()],
})


,raw_step_present_pct,canonical_step_present_pct,raw_but_not_canonical_pct,neither_raw_nor_canonical_pct
0,0.967742,0.677419,0.290323,0.032258


In [5]:
pd.DataFrame({
    "raw_step_present_pct": [has_raw_step.mean()],
    "canonical_step_present_pct": [has_canonical.mean()],
    "raw_but_not_canonical_pct": [(has_raw_step & ~has_canonical).mean()],
    "neither_raw_nor_canonical_pct": [(~has_raw_step & ~has_canonical).mean()],
}).to_dict(orient="records")

[{'raw_step_present_pct': 0.967741935483871,
  'canonical_step_present_pct': 0.6774193548387096,
  'raw_but_not_canonical_pct': 0.2903225806451613,
  'neither_raw_nor_canonical_pct': 0.03225806451612903}]

In [6]:
df_recruiting["canonical_current_step_match_type"].value_counts(dropna=False)


canonical_current_step_match_type
fuzzy    12
none     10
alias     9
Name: count, dtype: int64

In [7]:
failed = df_recruiting[has_raw_step & ~has_canonical]
failed["state.step"].value_counts().head(30)


state.step
candidate intake / search                           1
role understanding/assessment                       1
searches and candidate pipeline                     1
candidate outreach approval                         1
tech screen decision                                1
candidate follow-up / timing decision               1
HR interview coordination / challenge resolution    1
Confirm role scope / search expansion               1
notes/documentation shared                          1
Name: count, dtype: int64

In [8]:
matched = df_recruiting[has_canonical]
matched[["state.step","canonical_current_step_id","canonical_current_step_match_type"]].head(30)


,state.step,canonical_current_step_id,canonical_current_step_match_type
0,candidate outreach / interview scheduling,interview-scheduling,fuzzy
1,post-interview feedback,client-interview-feedback,alias
2,Client review / feedback on candidate CV and n...,client-profile-feedback,alias
3,awaiting feedback / rejection communicated,client-profile-feedback,fuzzy
5,awaiting candidate responses,interview-scheduling,alias
7,interview scheduled,interview-scheduling,alias
9,Client search on hold (awaiting reopening to p...,interview-scheduling,fuzzy
10,candidate selection,client-profile-feedback,alias
11,feedback shared,client-interview-feedback,alias
12,interview scheduling,interview-scheduling,alias


In [9]:
(df_recruiting.assign(has_canonical=has_canonical)
 .groupby("canonical_client")["has_canonical"]
 .mean()
 .sort_values(ascending=False)
 .head(20))


canonical_client
Celara Labs     1.000000
Kerry           1.000000
Public Relay    1.000000
Chromatics      0.692308
QU              0.000000
Qb              0.000000
Name: has_canonical, dtype: float64

In [10]:
val = "candidate selection"
rows = df_recruiting[df_recruiting["state.step"] == val]
rows[["instance_key", "evidence"]].head(3)


,instance_key,evidence
10,thread:199e91bdb002d6ca,"[{'message_id': 'gmail_199eb1ab091c4ebf', 'tim..."


In [11]:
rows.evidence.item()

[{'message_id': 'gmail_199eb1ab091c4ebf',
  'timestamp': '2025-10-16T00:23:14',
  'event_type': 'decision',
  'confidence': 0.74,
  'snippet': 'I don’t think guido or german will work'}]